In [1]:
from extractor import extractor
import numpy as np
import os
path_en = 'data/voxforge/english'
file_list = os.listdir(path_en)
'''
extractor.time_offset chooses which part of the recording to begin saving
extractor.snippet_length chooses the length of the recording snippet
'''
for f in range(len(file_list)):
    file_list[f] = path_en + '/' + file_list[f]
features_en = extractor.make_feature_set(file_list) 


/Users/oosv/miniconda3/lib/python3.6/site-packages/mkl_fft/_numpy_fft.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.rfft_numpy(a, n=n, axis=axis)


In [2]:
path_es = 'data/voxforge/spanish'
file_list = os.listdir(path_es)

for f in range(len(file_list)):
    file_list[f] = path_es + '/' + file_list[f]

features_es = extractor.make_feature_set(file_list) 


/Users/oosv/miniconda3/lib/python3.6/site-packages/mkl_fft/_numpy_fft.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.rfft_numpy(a, n=n, axis=axis)


In [3]:
import imp
from models import gmm
imp.reload(gmm)
single_model = gmm.GMM()


In [4]:
#only run if voxforge lines were used
import numpy as np
ready_es = single_model.adjust_data_to_model(features_es)
ready_en = single_model.adjust_data_to_model(features_en)
lang_to_id = {'english' : 0, 'spanish': 1}


In [5]:
import pandas as pd
en_shape = np.shape(ready_en)
es_shape = np.shape(ready_es)
ready_en = np.array(ready_en).reshape(en_shape[0] * en_shape[1], en_shape[2])
ready_es = np.array(ready_es).reshape(es_shape[0] * es_shape[1], es_shape[2])
print(np.shape(ready_en))
print(np.shape(ready_es))


(4939050, 39)
(1760400, 39)


In [6]:
print(np.shape(ready_en))
print(np.shape(ready_es))
df = pd.DataFrame(ready_en)
df.to_csv("english_data.csv", header = False, index = False)
df = pd.DataFrame(ready_es)
df.to_csv("spanish_data.csv", header = False, index = False)

(4939050, 39)
(1760400, 39)


In [36]:
print(f"total samples_en:\t{len(features_en)}")
print(f"total hours english:\t{len(features_en) * 0.75 / 60}")
print(f"total samples_es:\t{len(features_es)}")
print(f"total hours spanish:\t{len(features_es) * 0.75 / 60}")
print(f"total hours combined:\t{(len(features_es) + len(features_en)) * 0.75/60}")


total samples_en:	65854
total hours english:	823.175
total samples_es:	23472
total hours spanish:	293.4
total hours combined:	1116.575


In [196]:
train_en = ready_en[:int(len(ready_en) * 3 / 4)]
train_es = ready_es[:int(len(ready_es) * 3 / 4)]
test_en = ready_en[int(len(ready_en) * 3 / 4) + 1:]
test_es = ready_es[int(len(ready_es) * 3 / 4) + 1:]


In [169]:
print(f"total training samples_en:\t{len(train_en)}")
print(f"total training hours english:\t{len(train_en) * 0.75 / 3600}")
print(f"total training samples_es:\t{len(train_es)}")
print(f"total training hours spanish:\t{len(train_es) * 0.75 / 3600}")
print(f"total training hours combined:\t{(len(train_es) + len(train_en)) * 0.75 /3600}")


total training samples_en:	49390
total training hours english:	10.289583333333333
total training samples_es:	17604
total training hours spanish:	3.6675
total training hours combined:	13.957083333333333


In [168]:
print(f"shape train english: {np.shape(train_en)}")
print(f"shape train spanish: {np.shape(train_es)}")
print(f"shape test english: {np.shape(test_en)}")
print(f"shape test spanish: {np.shape(test_es)}")
print(f"total testing samples_en:\t{len(test_en)}")
print(f"total testing hours english:\t{len(test_en) * 0.75 / 3600}")
print(f"total testing samples_es:\t{len(test_es)}")
print(f"total testing hours spanish:\t{len(test_es) * 0.75 / 3600}")
print(f"total testing hours combined:\t{(len(test_en) + len(test_es)) * 0.75/3600}")


shape train english: (49390, 75, 39)
shape train spanish: (17604, 75, 39)
shape test english: (16463, 75, 39)
shape test spanish: (5867, 75, 39)
total testing samples_en:	16463
total testing hours english:	3.4297916666666666
total testing samples_es:	5867
total testing hours spanish:	1.2222916666666668
total testing hours combined:	4.652083333333334


In [182]:
#initiate gmm model 
#single model example
model = gmm.GMM(n_clusters=[30,30]) #There are other parameters but for simplicity, an array of clusters will do



In [194]:
gmm_models = [gmm.GMM(n_clusters=[50,50]), \
              gmm.GMM(n_clusters=[50,30]), \
              gmm.GMM(n_clusters=[30,30]), \
              gmm.GMM(n_clusters=[40,50]), \
              gmm.GMM(n_clusters=[40,40]), \
              gmm.GMM(n_clusters=[50,40]), \
              gmm.GMM(n_clusters=[30,40]), \
              gmm.GMM(n_clusters=[40,30])]


In [197]:
import pprint
predictions = {n : [] for n in range(len(gmm_models))}
for idx, gmm_model in enumerate(gmm_models):
    gmm_model.train(train_en[:1000], lang_to_id['english'])
    gmm_model.train(train_es[:1000], lang_to_id['spanish'])
    predictions[idx] = {'english' : \
                       np.average(np.array(gmm_model.predict_all(test_en[:50])) == lang_to_id['english']) , \
                       'spanish' : \
                       np.average(np.array(gmm_model.predict_all(test_es[:50])) == lang_to_id['spanish'])}
print(len(gmm_model.predict_all(test_en[:50])))
pprint.pprint(predictions)


50
{0: {'english': 0.42, 'spanish': 0.42},
 1: {'english': 0.18, 'spanish': 0.8},
 2: {'english': 0.34, 'spanish': 0.76},
 3: {'english': 0.48, 'spanish': 0.3},
 4: {'english': 0.34, 'spanish': 0.66},
 5: {'english': 0.26, 'spanish': 0.74},
 6: {'english': 0.46, 'spanish': 0.76},
 7: {'english': 0.24, 'spanish': 0.68}}


In [203]:
for model in gmm_models:
    print(f"english: {np.average(np.array(model.predict_all(test_en[:300])) == 0)}")
    print(f"spanish: {np.average(np.array(model.predict_all(test_es[:300])) == 1)}")
    

english: 0.35
spanish: 0.5233333333333333
english: 0.19
spanish: 0.8433333333333334
english: 0.32666666666666666
spanish: 0.7866666666666666
english: 0.47333333333333333
spanish: 0.5033333333333333
english: 0.34
spanish: 0.74
english: 0.26666666666666666
spanish: 0.8133333333333334
english: 0.4166666666666667
spanish: 0.7533333333333333
english: 0.2733333333333333
spanish: 0.8066666666666666


In [199]:
print(model.cluster_distributions)
                                

[defaultdict(<class 'float'>, {47: 0.026786666666666667, 46: 0.029786666666666666, 34: 0.025706666666666666, 22: 0.024586666666666666, 10: 0.031613333333333334, 3: 0.011546666666666667, 23: 0.01988, 32: 0.03392, 33: 0.020626666666666668, 42: 0.01572, 6: 0.008546666666666666, 27: 0.010253333333333333, 13: 0.01196, 4: 0.007533333333333334, 11: 0.016533333333333334, 29: 0.015466666666666667, 41: 0.012626666666666666, 31: 0.009746666666666667, 7: 0.008986666666666667, 39: 0.02432, 43: 0.023186666666666668, 14: 0.012693333333333333, 25: 0.01324, 1: 0.014666666666666666, 48: 0.008146666666666667, 20: 0.0084, 12: 0.006573333333333333, 16: 0.008506666666666666, 35: 0.013293333333333334, 45: 0.012253333333333333, 30: 0.0051333333333333335, 18: 0.010786666666666667, 28: 0.008626666666666666, 49: 0.024386666666666668, 15: 0.006586666666666667, 37: 0.0082, 5: 0.0132, 9: 0.026413333333333334, 19: 0.023613333333333333, 40: 0.006613333333333333, 2: 0.016346666666666666, 26: 0.00944, 44: 0.00973333333